In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
def detect_touching(landmarks):
    finger_landmarks = landmarks.landmark

    # Define landmarks for thumb and index fingers
    thumb_tip_id = 4
    index_tip_id = 8

    if len(finger_landmarks) >= 21:  # Check if enough landmarks are detected
        thumb_tip = np.array([finger_landmarks[thumb_tip_id].x, finger_landmarks[thumb_tip_id].y])
        index_tip = np.array([finger_landmarks[index_tip_id].x, finger_landmarks[index_tip_id].y])

        # Define a distance threshold for considering fingers as touching
        distance_threshold = 0.05  # Adjust this threshold as needed

        # Calculate the Euclidean distance between thumb and index fingertips
        distance = np.linalg.norm(thumb_tip - index_tip)

        # Check if the distance is less than the threshold
        if distance < distance_threshold:
            return True

    return False
# Set the frame size to 1280x720 (720p)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)

        # Convert the BGR image to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame and detect hands
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Detect if thumb and index fingers are touching
                is_touching = detect_touching(landmarks)

                if is_touching:
                    cv2.putText(frame, 'Touching', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Hand Tracking', frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
            break

cap.release()
cv2.destroyAllWindows()


